In [1]:
import os
import re
from pathlib import Path

import pandas as pd
from deep_translator import GoogleTranslator
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langdetect import detect
from mlx_lm import generate, load

/Users/sergey/Desktop/Moodle_RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

CHUNKS_DIR = Path(os.environ["MOODLE_CHUNKS_DIR"])
PERSIST_DIR = os.environ["MOODLE_CHROMA_DB_DIR"]
COLLECTION_NAME = os.environ.get("MOODLE_COLLECTION_NAME", "moodle_docs")

In [4]:
# Загрузка модели для получения эмбеддингов, маленькая и хорошо работает на CPU

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

In [5]:
# LLM для ответа
model, tokenizer = load("mlx-community/Qwen2.5-7B-Instruct-4bit")

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 102300.10it/s]


In [6]:
# Подключиться к уже сохраненной базе
vector_store = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=hf_embeddings,
    persist_directory=PERSIST_DIR,
)

print("Collection:", COLLECTION_NAME)
print("Documents in collection:", vector_store._collection.count())

Collection: moodle_docs
Documents in collection: 2697


In [7]:
# так как модель эмбеддингов работает только с английским переводим запрос на него если он на руссском


def prepare_query(user_query: str):
    text = (user_query or "").strip()
    if not text:
        return "", "ru"

    try:
        lang = detect(text)
    except Exception:
        lang = "ru"

    # Нормализация до ru/en
    if lang not in ("ru", "en"):
        lang = "en"  # или "ru", но тогда чаще будет русский ответ

    if lang == "ru":
        try:
            query_en = GoogleTranslator(source="ru", target="en").translate(text)
            if not query_en:
                query_en = text
        except Exception:
            query_en = text
    else:
        query_en = text

    return query_en, lang

In [8]:
# собираем контекст из топ-5 наиболее релевантных документов, возвращаем его вместе с языком запроса и результатами поиска для дальнейшего использования в ответе модели
# Технически можно подавать только doc.page_content, но качество и управляемость обычно хуже.


def build_context(user_query: str, vector_store, k: int = 5):
    query_en, user_lang = prepare_query(user_query)
    results = vector_store.similarity_search_with_score(query_en, k=k)

    context_blocks = []
    for i, (doc, score) in enumerate(results, 1):
        distance = float(score)  # чем меньше, тем релевантнее
        context_blocks.append(
            f"[{i}]\n"
            f"doc_title: {doc.metadata.get('doc_title', 'unknown')}\n"
            f"distance: {distance:.4f}\n"
            f"source_links: {doc.metadata.get('source_links', [])}\n"
            f"youtube_links: {doc.metadata.get('youtube_links', [])}\n"
            f"text:\n{doc.page_content}"
        )

    context = "\n\n---\n\n".join(context_blocks)
    return context, user_lang, results

In [9]:
def generate_answer(user_query: str, context: str, user_lang: str, recent_history=None):
    answer_lang = "Russian" if user_lang == "ru" else "English"

    system_prompt = f"""
    <ROLE_DEFINITION>
    You are the official Moodle documentation assistant acting as a technical consultant.
    </ROLE_DEFINITION>

    <MAIN_TASK_GUIDELINES>
    Your task is to provide precise, formal, and verifiable answers strictly based on the provided CONTEXT.
    You must directly answer the user’s question without adding external knowledge.
    If the answer is not present in the CONTEXT, respond exactly: "Not found in the documentation".
    Do not make assumptions, interpretations, or extrapolations beyond the CONTEXT.
    The response must be structured and concise.
    </MAIN_TASK_GUIDELINES>

    <IMPORTANT_LANGUAGE_GUIDELINES>
    Determine the language of the user's query and use THAT SAME language for:
    - all actions,
    - all search formulations,
    - the final answer,
    - all textual fields and outputs.

    Answer strictly in {answer_lang}.

    If the query is in Russian — all fields and responses must be strictly in Russian.
    If the query is in English — all fields and responses must be strictly in English.
    </IMPORTANT_LANGUAGE_GUIDELINES>

    <OUTPUT_FORMAT_REQUIREMENTS>
    The ending section is mandatory and must always be included:

    - source_links: [list of links from CONTEXT]
    - youtube_links: [list of links from CONTEXT if available; if none — write "none"]
    </OUTPUT_FORMAT_REQUIREMENTS>
    """.strip()

    messages = [{"role": "system", "content": system_prompt}]

    if recent_history:
        for m in recent_history:
            messages.append({"role": m["role"], "content": m["content"]})

    # Текущий вопрос + контекст
    messages.append(
        {
            "role": "user",
            "content": (
                f"QUESTION:\n{user_query}\n\n"
                f"CONTEXT:\n{context}\n\n"
                f"IMPORTANT: Respond ONLY in {answer_lang}. "
                f"If QUESTION is English -> English only. If Russian -> Russian only."
            ),
        }
    )

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    answer = generate(model, tokenizer, prompt=prompt, max_tokens=550)
    return answer

In [10]:
# Опционально: простая поддержка follow-up в retrieval


def retrieval_query_with_context(query: str, chat_history: list):
    q = query.strip()
    if len(q.split()) <= 4:
        prev_users = [m["content"] for m in chat_history if m["role"] == "user"]
        if prev_users:
            return prev_users[-1] + "\n" + q
    return q

In [11]:
def continual_chat(k: int = 5, history_turns: int = 3):
    print("Начните общение с ИИ! Введите 'выход', чтобы завершить разговор.")
    chat_history = []

    while True:
        query = input("ВЫ: ").strip()
        if query.lower() == "выход":
            break

        recent_history = chat_history[-2 * history_turns :]

        # retrieval с учетом контекста (без перефраза)
        rq = retrieval_query_with_context(query, chat_history)
        context, _, results = build_context(rq, vector_store, k=k)

        # язык ответа по текущему вопросу пользователя
        _, user_lang = prepare_query(query)

        # генерация с историей
        answer = generate_answer(query, context, user_lang, recent_history=recent_history)

        print("\n✅ Ответ AI:")
        print(answer)

        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": answer})


if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ! Введите 'выход', чтобы завершить разговор.

✅ Ответ AI:
Чтобы создать новый курс в Moodle, выполните следующие шаги:

1. Перейдите в раздел Site administration > Courses > Manage courses and categories.
2. Нажмите на ссылку "New course" в категории, где вы хотите создать новый курс.
3. Введите настройки курса, затем выберите "Save and return" для возврата к своей странице курса или "Save and display" для перехода к следующему экрану.
4. На следующем экране, если вы выбрали "Save and display", назначьте студентов/преподавателей для курса.

Источники:
- [https://docs.moodle.org/403/en/Adding_a_new_course](https://docs.moodle.org/403/en/Adding_a_new_course)
- [https://youtu.be/MzK2jb-9SwE](https://youtu.be/MzK2jb-9SwE)

source_links: ['https://docs.moodle.org/403/en/Adding_a_new_course', 'https://youtu.be/MzK2jb-9SwE']
youtube_links: ['https://youtu.be/MzK2jb-9SwE']

✅ Ответ AI:
Чтобы настроить систему оценок в Moodle, выполните следующие шаги:

1. Перейдите в раздел

# Максимально простой замер базовых метрик RAGа

In [ ]:
# Максимально простой eval-блок:
# retrieval: Hit@k + MRR
# generation: LLM-as-judge faithfulness (с цитатами)
# attribution: exact source attribution check

# Небольшой eval-набор:
eval_items = [
    {
        "question": "Как создать новый курс в Moodle?",
        "relevant_sources": ["https://docs.moodle.org/403/en/Adding_a_new_course"],
    },
    {
        "question": "Как настроить систему оценок в Moodle?",
        "relevant_sources": ["https://docs.moodle.org/403/en/Using_Assignment"],
    },
    {
        "question": "Как просмотреть журналы активности пользователей?",
        "relevant_sources": ["https://docs.moodle.org/403/en/Activity_report"],
    },
]


def _to_list(x):
    if x is None:
        return []
    if isinstance(x, list):
        return [str(i).strip() for i in x if str(i).strip()]
    s = str(x).strip()
    return [s] if s else []


def get_ranked_source_links(results):
    # results: [(doc, score), ...] из similarity_search_with_score
    ranked = []
    for doc, _ in results:
        ranked.append(_to_list(doc.metadata.get("source_links", [])))
    return ranked


def retrieval_hit_mrr(results, relevant_sources, k=5):
    ranked = get_ranked_source_links(results)[:k]
    relevant = set(relevant_sources)

    hit = 0
    mrr = 0.0

    for rank, links in enumerate(ranked, start=1):
        if any(link in relevant for link in links):
            hit = 1
            mrr = 1.0 / rank
            break

    return hit, mrr


def extract_urls(text):
    return re.findall(r"https?://[^\s\]\),]+", text or "")


def source_attribution_check(answer, results):
    answer_links = set(extract_urls(answer))
    retrieved_links = set()
    for links in get_ranked_source_links(results):
        for l in links:
            retrieved_links.add(l)

    all_cited_from_retrieved = len(answer_links) > 0 and answer_links.issubset(retrieved_links)
    return all_cited_from_retrieved, sorted(answer_links), sorted(retrieved_links)


def llm_judge_faithfulness(question, context, answer):
    judge_prompt = f"""
You are a strict faithfulness judge.
Task: check whether ANSWER is fully supported by CONTEXT only.

Return EXACTLY in this format:
FAITHFUL: yes/no
CITATIONS:
- "short quote 1 from CONTEXT"
- "short quote 2 from CONTEXT"

Rules:
- If any claim in ANSWER is not in CONTEXT => FAITHFUL: no
- Quotes must be verbatim from CONTEXT
- Max 2 quotes

QUESTION:
{question}

CONTEXT:
{context}

ANSWER:
{answer}
""".strip()

    messages = [{"role": "user", "content": judge_prompt}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    judge_raw = generate(model, tokenizer, prompt=prompt, max_tokens=180)

    faithful = 1 if "FAITHFUL: yes" in judge_raw else 0
    citations = re.findall(r'-\s*"([^"]+)"', judge_raw)
    return faithful, citations, judge_raw


# 2) Запуск
rows = []

for item in eval_items:
    q = item["question"]
    relevant = item["relevant_sources"]

    context, user_lang, results = build_context(q, vector_store, k=5)
    answer = generate_answer(q, context, user_lang)

    hit5, mrr5 = retrieval_hit_mrr(results, relevant, k=5)
    faithful, judge_citations, judge_raw = llm_judge_faithfulness(q, context, answer)
    exact_attr_ok, answer_links, retrieved_links = source_attribution_check(answer, results)

    rows.append(
        {
            "question": q,
            "hit@5": hit5,
            "mrr@5": round(mrr5, 4),
            "faithfulness_llm_judge": faithful,  # 1/0
            "faithfulness_citations": judge_citations,
            "exact_source_attribution_ok": exact_attr_ok,
            "answer_links": answer_links,
            "relevant_sources": relevant,
            "answer": answer,
        }
    )

df_eval = pd.DataFrame(rows)
df_eval

,question,hit@5,mrr@5,faithfulness_llm_judge,faithfulness_citations,exact_source_attribution_ok,answer_links,relevant_sources,answer
0,Как создать новый курс в Moodle?,1,1.0,1,[By default a regular teacher can't add a new ...,False,[https://docs.moodle.org/403/en/Adding_a_new_c...,[https://docs.moodle.org/403/en/Adding_a_new_c...,"Чтобы создать новый курс в Moodle, выполните с..."
1,Как настроить систему оценок в Moodle?,1,1.0,1,[Click on the assignment name on the Moodle co...,False,[https://docs.moodle.org/403/en/Grading_quick_...,[https://docs.moodle.org/403/en/Using_Assignment],"Чтобы настроить систему оценок в Moodle, выпол..."
2,Как просмотреть журналы активности пользователей?,1,1.0,1,[Administration > Course administration > Repo...,True,[https://docs.moodle.org/403/en/Activity_repor...,[https://docs.moodle.org/403/en/Activity_report],Чтобы просмотреть журналы активности пользоват...
